<a href="https://colab.research.google.com/github/zhgjenny93/NLP-Thinkful/blob/main/Text_Preprocessing_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment, you'll clean up the two datasets. You'll be using these datasets in the later checkpoints of this module, and cleaning them up here will help you save time when working with these datasets.

The first dataset is a dialogue dataset called [Cornell Movie--Dialogs Corpus](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). This corpus includes conversations between the characters of more than 600 movies.

The second dataset is the [Twitter US Airline Sentiment](https://www.kaggle.com/crowdflower/twitter-airline-sentiment) dataset from Kaggle. This dataset contains tweets from travelers about some airlines in February 2015. This dataset is usually used in sentiment analysis, but you'll use it for sentence generation later on.

**Note:** Because the memory requirements of the datasets are relatively large, it's best to use Google Colaboratory for this assignment.

Submit your solutions to the following tasks as a link to your Jupyter Notebook on GitHub.

1. Apply the data preprocessing techniques that you learned here to the Cornell Movie--Dialogs Corpus data. You'll use this dataset when developing a chatbot in an upcoming checkpoint. Access the dataset from the Thinkful database using the following credentials:

 ```
 postgres_user = 'dsbc_student'
 postgres_pw = '7*.8G9QH21'
 postgres_host = '142.93.121.174'
 postgres_port = '5432'
 postgres_db = 'cornell_movie_dialogs'
 ```
The data is in the table called dialogs.

2. Apply the data preprocessing techniques that you learned here to Twitter US Airline Sentiment data. You'll use this dataset when generating sentences in an upcoming checkpoint. You should access the dataset from the Thinkful database using the following credentials:

 ```
 postgres_user = 'dsbc_student'
 postgres_pw = '7*.8G9QH21'
 postgres_host = '142.93.121.174'
 postgres_port = '5432'
 postgres_db = 'twitter_sentiment'
 ```
The data is in the table called twitter.

**Note:** When parsing the data using spaCy, you may run into some memory issues, even in Google Colaboratory. If you're having memory issues, try parsing your text as follows:

```
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 20000000
doc = nlp(the_dialogs_come_here)
```

## Part 1

In [ ]:
from sqlalchemy import create_engine
import nltk
import spacy
import re
import pandas as pd
import numpy as np
from collections import Counter

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

dialogs_df = pd.read_sql_query('select * from dialogs', con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

dialogs_df.head(10)

,index,dialogs
0,0,Can we make this quick? Roxanne Korrine and A...
1,1,"Well, I thought we'd start with pronunciation,..."
2,2,Not the hacking and gagging and spitting part....
3,3,Okay... then how 'bout we try out some French ...
4,4,You're asking me out. That's so cute. What's ...
5,5,Forget it.
6,6,"No, no, it's my fault -- we didn't have a prop..."
7,7,Cameron.
8,8,"The thing is, Cameron -- I'm at the mercy of a..."
9,9,Seems like she could get a date easy enough...


In [ ]:
# Remove new lines and other extra whitespace by creating str from df and splitting and rejoining

dialogs_str = ' '.join(dialogs_df.dialogs)
dialogs_str = ' '.join(dialogs_str.split())

In [ ]:
print(dialogs_str[:300])

Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again. Well, I thought we'd start with pronunciation, if that's okay with you. Not the hacking and gagging and spitting part. Please. Okay... then how 'bout we try out some Fr


In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done here, may take a while
dialogs_doc = nlp(dialogs_str)

In [ ]:
print('The dialogs_doc object is a {} object'.format(type(dialogs_doc)))
print('It is {} tokens long'.format(len(dialogs_doc)))
print("The first three tokens are '{}'".format(dialogs_doc[:3]))
print("The type of each token is {}".format(type(dialogs_doc[0])))

The dialogs_doc object is a <class 'spacy.tokens.doc.Doc'> object
It is 4189028 tokens long
The first three tokens are 'Can we make'
The type of each token is <class 'spacy.tokens.token.Token'>


In [ ]:
# Removing stopwords
dialogs_without_stopwords = [token for token in dialogs_doc if not token.is_stop]

In [ ]:
# print(dialogs_without_stopwords[:10])

In [ ]:
# Utility function to calculate how frequently words appear in the text
def word_frequencies(text):
  # Build a list of words
  # Strip out punctuation
  words=[]
  for token in text:
    if not token.is_punct:
      words.append(token.text)
  
  # Build and return a `Counter` object containing word counts
  return Counter(words)

# Instantiate list of the most common words
dialogs_word_freq = word_frequencies(dialogs_without_stopwords).most_common(10)
print(dialogs_word_freq)

[('know', 21478), ('like', 13765), ('got', 12663), ('want', 10800), ('think', 10427), ('going', 8770), ('right', 8710), ('>', 7669), ('Oh', 7516), ('time', 6452)]


In [ ]:
# lemmatization
lemmas = [token.lemma_ for token in dialogs_without_stopwords]

In [ ]:
# Utility function to calculate how frequently each lemma appears in the text
def lemma_frequencies(text):
  # Build a list of lemmas
  # Strip out punctuation
  lemmas = []
  for token in text:
    if not token.is_punct:
      lemmas.append(token.lemma_)
    
  # Build and return a `Counter` object containing word counts
  return Counter(lemmas)

# Instantiate your list of the most common words
dialogs_lemma_freq = lemma_frequencies(dialogs_without_stopwords).most_common(10)

print(dialogs_lemma_freq)

[('know', 24746), ('go', 16862), ('like', 15622), ('get', 15430), ('think', 15067), ('want', 13978), ('come', 10594), ('tell', 10492), ('right', 10108), ('look', 8811)]


## Part 2

In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

tweets_df = pd.read_sql_query('select * from twitter',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


tweets_df.head(10)

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)
5,5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,None,jnardino,None,0,@VirginAmerica seriously would pay $30 a fligh...,None,2015-02-24 11:14:33 -0800,None,Pacific Time (US & Canada)
6,6,570300616901320704,positive,0.6745,None,0.0000,Virgin America,None,cjmcginnis,None,0,"@VirginAmerica yes, nearly every time I fly VX...",None,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,7,570300248553349120,neutral,0.6340,None,NaN,Virgin America,None,pilot,None,0,@VirginAmerica Really missed a prime opportuni...,None,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,8,570299953286942721,positive,0.6559,None,NaN,Virgin America,None,dhepburn,None,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",None,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,9,570295459631263746,positive,1.0000,None,NaN,Virgin America,None,YupitsTate,None,0,"@VirginAmerica it was amazing, and arrived an ...",None,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


In [ ]:
nlp = spacy.load('en')

nlp.max_length = 20000000

twitter_doc = nlp(" ".join(tweets_df.text))

In [ ]:
# let's explore the objects we've built
print("The twitter_doc object is a {} object".format(type(twitter_doc)))
print("It is {} tokens long".format(len(twitter_doc)))
print("The first three tokens are '{}'".format(twitter_doc[:3]))
print("The type of each token is {}".format(type(twitter_doc[0])))

The twitter_doc object is a <class 'spacy.tokens.doc.Doc'> object
It is 307328 tokens long
The first three tokens are '@VirginAmerica What @dhepburn'
The type of each token is <class 'spacy.tokens.token.Token'>


In [ ]:
# Removing the stopwords
tweets_without_stopwords = [token for token in twitter_doc if not token.is_stop]

tweets_word_freq = word_frequencies(tweets_without_stopwords).most_common(10)

In [ ]:
print(tweets_word_freq)

[('@united', 3733), ('flight', 3178), ('@AmericanAir', 2904), ('@USAirways', 2893), ('@SouthwestAir', 2390), ('@JetBlue', 2176), (' ', 1951), ('Cancelled', 1065), ('service', 928), ('time', 770)]


In [ ]:
# lemmatization
tweet_lemmas = [token.lemma_ for token in tweets_without_stopwords]

tweets_lemma_freq = lemma_frequencies(tweets_without_stopwords).most_common(10)

In [ ]:
print(tweets_lemma_freq)

[('flight', 4006), ('@unite', 2712), ('@usairways', 2616), (' ', 1951), ('@JetBlue', 1740), ('thank', 1654), ('@AmericanAir', 1489), ('@americanair', 1450), ('@southwestair', 1280), ('@SouthwestAir', 1157)]
